term frequency

In [ ]:
!pip install gensim

In [ ]:
import gensim as gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pandas as pd
import os

In [ ]:

def iter_documents(top_directory):
    """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
    for root, dirs, files in os.walk(top_directory):
        for file in filter(lambda file: file.endswith('.txt'), files):
            document = open(os.path.join(root, file)).read() # read the entire document, as one big string
            yield gensim.utils.tokenize(document, lower=True) # or whatever tokenization suits you

class MyCorpus(object):
    def __init__(self, top_dir):
        self.top_dir = top_dir
        self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
#         self.dictionary.filter_extremes(no_below=1, keep_n=30000) # check API docs for pruning params

    def __iter__(self):
        for tokens in iter_documents(self.top_dir):
            yield self.dictionary.doc2bow(tokens)

corpus = MyCorpus('/home/jovyan/work/resources/data_ignored/tf/RDU0015_Half_jaar_verslagen_voor_NLP')

In [ ]:
dict_token = corpus.dictionary.token2id
dict_cfs = corpus.dictionary.cfs

tf_list = []
for key in dict_token:
    tf_list.append((key, dict_cfs[dict_token[key]]))

In [ ]:
print(tf_list[0])

In [ ]:
df = pd.DataFrame(tf_list, columns= ["term", "frequency"])

df.to_excel('/home/jovyan/work/resources/data_ignored/tf/RDU0015_Half_jaar_verslagen_voor_NLP/tf.xlsx')

In [ ]:
print(df.head(20))

In [ ]:
xls = pd.ExcelFile('/home/jovyan/work/resources/data_ignored/tf/list_topix.xlsx')
df1 = pd.read_excel(xls, 'Zoekhulp Rad. ')
df2 = pd.read_excel(xls, 'Zoekhulp MED.')

In [ ]:
list_rad = df1['Radiologie topic (00.20.13.01)'].to_list()
list_med = df2['Algemeen medisch topic 00.20.13.01'].to_list()

In [ ]:
list_rad_lower = [str(x).lower() for x in list_rad]
list_med_lower = [str(x).lower() for x in list_med]

In [ ]:
list_term = list_rad_lower + list_med_lower

In [ ]:
len(list_term)

In [ ]:
df_filtered = df[~df["term"].isin(list_term)]

In [ ]:
print(len(df), len(df_filtered))

In [ ]:
df_filtered.to_excel('/home/jovyan/work/resources/data_ignored/tf/RDU0015_Half_jaar_verslagen_voor_NLP/tf_filtered.xlsx')